# Ground-truth performance and gradient norms

This notebook inspects the initialization schemes for LQGs and time-varying linear policies described in section 5.1 of the [paper](https://www.overleaf.com/read/cmbgmxxpxqzr).

### Checklist

- [x] Fix `n_state`, `n_ctrl`, `horizon`
- [x] Sample random LQGs
- [x] Sample random policies
- [x] Evaluate the expected return
- [x] Evaluate the value gradient norm
- [x] Search numpy, scipy for methods for visualizing the distributions

---
## Linear Quadratic Gaussian (LQG) Problems
In what follows we consider MDPs with:
1. continuous state space $\mathbf{s} \in \mathcal{S} = \mathbb{R}^n$
2. continuous action space $\mathbf{a} \in \mathcal{A} = \mathbb{R}^d$
3. finite time horizon $N \in \mathbb{N}$ and timesteps $t \in \mathcal{T} = \{0, \dots, N - 1\}$
4. time-varying linear Gaussian dynamics 
    $$
    \mathbf{s}_{t+1} \sim p(\cdot| \mathbf{s}_t, \mathbf{a}_t) = \mathcal{N}\left( \cdot ~\middle|~ \mathbf{F}_t \begin{bmatrix}\mathbf{s}_t \\ \mathbf{a}_t\end{bmatrix} + \mathbf{f}_t, \mathbf{\Sigma}_{t} \right)
    $$
5. time-varying quadratic costs 
    $$
    r_{t+1} = R(\mathbf{s}_t, \mathbf{a}_t) = - \tfrac{1}{2} \begin{bmatrix}\mathbf{s}_t \\ \mathbf{a}_t\end{bmatrix}^\intercal \mathbf{C}_t \begin{bmatrix}\mathbf{s}_t \\ \mathbf{a}_t\end{bmatrix} - \mathbf{c}_t^\intercal \begin{bmatrix}\mathbf{s}_t \\ \mathbf{a}_t\end{bmatrix}
    $$
6. Gaussian-distributed initial state 
    $$
    \mathbf{s}_0 \sim \rho = \mathcal{N}(\mathbf{\mu}_\rho, \mathbf{\Sigma}_\rho)
    $$

### Imports

In [1]:
%matplotlib inline
from __future__ import annotations

import textwrap
import warnings
from functools import partial
from typing import Callable, Optional

import lqsvg.torch.named as nt
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import torch
import torch.nn as nn
from lqsvg.envs import lqr
from lqsvg.envs.lqr.gym import LQGGenerator
from lqsvg.experiment.models import ExpectedValue, PolicyLoss
from lqsvg.experiment.utils import linear_feedback_distance, linear_feedback_norm
from matplotlib import cm
from mpl_toolkits.mplot3d import Axes3D
from scipy.stats import ortho_group
from torch import Tensor
from tqdm.notebook import tqdm

sns.set()

---
## Task selection protocol

$$
    \{ \mathbf{F}_t, \mathbf{f}_t, \mathbf{\Sigma}_t, \mathbf{C}_t, \mathbf{c}_t  \}_{t\in\mathcal{T}} \sim \text{LQGDist}(\texttt{n_state, n_ctrl, horizon}) \\
    \mathbf{\mu}_\rho = \mathbf{0}, \quad \mathbf{\Sigma}_{\rho} = \mathbf{I} \\
    \mathcal{M} = \{ \mathbf{F}_t, \mathbf{f}_t, \mathbf{\Sigma}_t, \mathbf{C}_t, \mathbf{c}_t  \}_{t\in\mathcal{T}} \cup \{ \mathbf{\mu}_\rho, \mathbf{\Sigma}_\rho \}
$$

In [5]:
def sample_lqg(
    n_state: int, n_ctrl: int, horizon: int, n_batch: Optional[int] = None
) -> tuple[lqr.LinSDynamics, lqr.QuadCost, lqr.GaussInit]:
    generator = LQGGenerator(
        n_state,
        n_ctrl,
        horizon,
        stationary=True,
        Fs_eigval_range=(0.0, 2.0),
        transition_bias=False,
        rand_trans_cov=True,
        rand_init_cov=False,
        cost_linear=False,
        cost_cross=False,
    )
    dynamics, cost, init = generator(n_batch)

    F_s, F_a = nt.split(dynamics.F, [n_state, n_ctrl], dim="C")
    F_a = F_a / (torch.linalg.norm(nt.unnamed(F_a), dim=(-2, -1), keepdim=True) + 1e-8)
    dynamics = lqr.LinSDynamics(
        F=torch.cat([F_s, F_a], dim="C"), f=dynamics.f, W=dynamics.W
    )

    return dynamics, cost, init

In [6]:
def print_lqg(dynamics, cost, init):
    msg = f"""\
        dynamics:
            F: {dynamics.F.shape}; {dynamics.F.names}; min={dynamics.F.min()}; max={dynamics.F.max()}
            f: {dynamics.f.shape}; {dynamics.f.names}; min={dynamics.f.min()}; max={dynamics.f.max()}
            Sig: {dynamics.W.shape}; {dynamics.W.names}
        cost:
            C: {cost.C.shape}; {cost.C.names}            
            c: {cost.c.shape}; {cost.c.names}
        init:
            mu: {init.mu.shape}; {init.mu.names}            
            sig: {init.sig.shape}; {init.sig.names}\
        """
    print(textwrap.dedent(msg))
    if "B" not in cost.C.names:
        print("C_0:")
        print(textwrap.indent(str(cost.C.select("H", 0).numpy()), " " * 4))


def test_sample_lqg(n_batch=None):
    with warnings.catch_warnings():
        warnings.filterwarnings("ignore", message=".*Named tensors.*", module="torch")
        dynamics, cost, init = sample_lqg(
            n_state=2, n_ctrl=2, horizon=100, n_batch=n_batch
        )
    assert isinstance(dynamics, lqr.LinSDynamics)
    assert isinstance(cost, lqr.QuadCost)
    assert isinstance(init, lqr.GaussInit)
    print_lqg(dynamics, cost, init)


def test_batch_lqgs():
    test_sample_lqg(n_batch=10)

In [7]:
test_sample_lqg()

dynamics:
    F: torch.Size([100, 2, 4]); ('H', 'R', 'C'); min=-1.213716745376587; max=0.16405349969863892
    f: torch.Size([100, 2]); ('H', 'R'); min=0.0; max=0.0
    Sig: torch.Size([100, 2, 2]); ('H', 'R', 'C')
cost:
    C: torch.Size([100, 4, 4]); ('H', 'R', 'C')            
    c: torch.Size([100, 4]); ('H', 'R')
init:
    mu: torch.Size([2]); ('R',)            
    sig: torch.Size([2, 2]); ('R', 'C')        
C_0:
    [[ 2.274646    2.0066602   0.          0.        ]
     [ 2.0066602   2.6473014   0.          0.        ]
     [ 0.          0.          0.778606   -0.0830318 ]
     [ 0.          0.         -0.0830318   0.76030445]]


In [8]:
test_batch_lqgs()

dynamics:
    F: torch.Size([100, 10, 2, 4]); ('H', 'B', 'R', 'C'); min=-1.8809062242507935; max=0.8916661143302917
    f: torch.Size([100, 10, 2]); ('H', 'B', 'R'); min=0.0; max=0.0
    Sig: torch.Size([100, 10, 2, 2]); ('H', 'B', 'R', 'C')
cost:
    C: torch.Size([100, 10, 4, 4]); ('H', 'B', 'R', 'C')            
    c: torch.Size([100, 10, 4]); ('H', 'B', 'R')
init:
    mu: torch.Size([10, 2]); ('B', 'R')            
    sig: torch.Size([10, 2, 2]); ('B', 'R', 'C')        


---
## Policy selection protocol

$$
    \mu_\theta(\mathbf{s}) = \mathbf{a} \\
    \mathcal{M} \sim \text{LQGDist}(\texttt{n_state, n_ctrl, horizon}) \\
    \theta = \{ \mathbf{K}_t, \mathbf{k}_t \}_{t\in\mathcal{T}} \sim \text{PiDist}(\mathcal{M})
$$

In [9]:
def solution(
    dynamics: lqr.LinSDynamics, cost: lqr.QuadCost
) -> tuple[lqr.Linear, lqr.Quadratic, lqr.Quadratic]:
    n_state, n_ctrl, horizon = lqr.dims_from_dynamics(dynamics)
    solver = lqr.NamedLQGControl(n_state, n_ctrl, horizon)
    pistar, qstar, vstar = solver(dynamics, cost)
    return pistar, qstar, vstar


def optimal_policy(dynamics: lqr.LinSDynamics, cost: lqr.QuadCost) -> lqr.Linear:
    pistar, _, _ = solution(dynamics, cost)
    return pistar


def perturb_policy(policy: lqr.Linear) -> lqr.Linear:
    n_state, n_ctrl, _ = lqr.dims_from_policy(policy)
    K, k = (g + 0.5 * torch.randn_like(g) / (n_state + np.sqrt(n_ctrl)) for g in policy)
    #     scale = 1 / (n_state + np.sqrt(n_ctrl))
    #     K, k = (g + 2 * torch.rand_like(g) * scale - scale for g in policy)
    return (K, k)


def sample_policy(dynamics: lqr.LinSDynamics, cost: lqr.QuadCost) -> lqr.Linear:
    n_state, n_ctrl, horizon = lqr.dims_from_dynamics(dynamics)
    K = torch.empty((horizon,) + dynamics.F.shape[1:-2] + (n_ctrl, n_state))
    k = torch.empty((horizon,) + dynamics.F.shape[1:-2] + (n_ctrl,))
    nn.init.xavier_uniform_(K)
    nn.init.constant_(k, 0)
    random = (nt.horizon(nt.matrix(K)), nt.horizon(nt.vector(k)))

    #     pistar = optimal_policy(dynamics, cost)
    #     random = perturb_policy(pistar)
    return random

In [10]:
def print_policy(policy: lqr.Linear):
    K, k = policy
    print(
        f"""
        K: {K.shape}; {K.names}
        k: {k.shape}; {k.names}
    """
    )


def test_optimal_policy():
    dynamics, cost, _ = sample_lqg(n_state=2, n_ctrl=2, horizon=100, n_batch=10)
    print_policy(optimal_policy(dynamics, cost))


def test_sample_policy():
    dynamics, cost, _ = sample_lqg(n_state=2, n_ctrl=2, horizon=100, n_batch=10)
    print_policy(sample_policy(dynamics, cost))

In [11]:
test_optimal_policy()


        K: torch.Size([100, 10, 2, 2]); ('H', None, 'R', 'C')
        k: torch.Size([100, 10, 2]); ('H', None, 'R')
    


In [12]:
test_sample_policy()


        K: torch.Size([100, 10, 2, 2]); ('H', None, 'R', 'C')
        k: torch.Size([100, 10, 2]); ('H', None, 'R')
    


In [13]:
def policy_distance(policy_a: lqr.Linear, policy_b: lqr.Linear) -> Tensor:
    return linear_feedback_distance(policy_a, policy_b)

In [14]:
def test_policy_distance():
    dynamics, cost, _ = sample_lqg(n_state=10, n_ctrl=10, horizon=100, n_batch=10)
    optimal = optimal_policy(dynamics, cost)
    random = perturb_policy(optimal)
    distance = policy_distance(optimal, random)
    print(
        f"""\
    Policy distances: {distance}, {distance.shape}\
    """
    )

In [15]:
test_policy_distance()

    Policy distances: tensor([3.9913, 3.9705, 4.0332, 3.9826, 3.9669, 3.9685, 3.9635, 3.9673, 3.9614,
        4.0183]), torch.Size([10])    


---
## Ground-truth policy performance

<center><b>Sampling</b></center>
$$
    \mathcal{M} \sim \text{LQGDist}(\texttt{n_state, n_ctrl, horizon}) \\
    \theta = \{ \mathbf{K}_t, \mathbf{k}_t \}_{t\in\mathcal{T}} \sim \text{PiDist}(\mathcal{M})
$$
<br>

<center><b>Prediction</b></center>
<!-- LQG prediction can be seen as a function mapping policy parameters (with the dynamics kept constant) to value function coefficients: -->
$$
\left( \mathbf{Q}, \mathbf{q}, q, \mathbf{V}, \mathbf{v}, v \right) = \text{LQGPrediction}(\theta) \\
V^\mu(\mathbf{s}, t) = -\tfrac12 \mathbf{s}^\intercal \mathbf{V}_t(\theta) \mathbf{s} - \mathbf{v}_t(\theta)^\intercal \mathbf{s} - v_t(\theta)
$$
<br>

<center><b>Performance</b></center>
<!-- We can then express policy performance as a direct function of policy parameters: -->
$$
\begin{align}
J(\theta) 
    &= -\mathbb{E}_{\mathbf{s}\sim\rho} \left[ \tfrac12 \mathbf{s}^\intercal \mathbf{V}_0(\theta) \mathbf{s} + \mathbf{v}_0(\theta)^\intercal \mathbf{s} + v_0(\theta) \right] \\
    &= -\text{Tr}(\mathbf{V}_0(\theta)\mathbf{\Sigma}_0) - \mathbf{\mu}_0^\intercal \mathbf{V}_0(\theta) \mathbf{\mu}_0 - \mathbf{v}_0(\theta)^\intercal \mathbf{\mu}_0 - v_0(\theta)
\end{align}
$$

In [16]:
def policy_loss(
    policy: lqr.Linear,
    dynamics: lqr.LinSDynamics,
    cost: lqr.QuadCost,
    init: lqr.GaussInit,
) -> Tensor:
    n_state, n_ctrl, horizon = lqr.dims_from_dynamics(dynamics)
    loss_fn = PolicyLoss(n_state, n_ctrl, horizon)
    loss = loss_fn(policy, dynamics, cost, init)
    return loss


def policy_performance(
    policy: lqr.Linear,
    dynamics: lqr.LinSDynamics,
    cost: lqr.QuadCost,
    init: lqr.GaussInit,
) -> Tensor:
    return -policy_loss(policy, dynamics, cost, init)

In [17]:
def test_policy_loss():
    dynamics, cost, init = sample_lqg(n_state=2, n_ctrl=2, horizon=20, n_batch=10)
    policy = sample_policy(dynamics, cost)
    loss = policy_loss(policy, dynamics, cost, init)
    print(
        f"""\
    Loss: {loss}, ({loss.dtype}), ({loss.shape}); 
    ExpectedReturn: {-loss}, ({(-loss).dtype}), ({loss.shape})\
    """
    )

In [18]:
test_policy_loss()

    Loss: tensor([1.8992e+07, 3.2380e+03, 1.4692e+06, 5.8283e+08, 2.0671e+06, 1.2380e+04,
        1.2105e+08, 2.7014e+03, 1.4148e+11, 1.2227e+02], names=('B',)), (torch.float32), (torch.Size([10])); 
    ExpectedReturn: tensor([-1.8992e+07, -3.2380e+03, -1.4692e+06, -5.8283e+08, -2.0671e+06,
        -1.2380e+04, -1.2105e+08, -2.7014e+03, -1.4148e+11, -1.2227e+02],
       names=('B',)), (torch.float32), (torch.Size([10]))    


---
## Ground-truth value gradient norm

1. **Sampling**$\rightarrow \mathcal{M}, \theta$; 
2. **Prediction**$\rightarrow \left( \mathbf{Q}, \mathbf{q}, q, \mathbf{V}, \mathbf{v}, v \right)$; 
3. **Performance**$\rightarrow J(\theta)$;
4. **SVG norm**$\rightarrow \| \nabla J(\theta) \|$.

In [ ]:
def compute_batched_svg_norm(exp_ret: Tensor, policy: lqr.Linear) -> Tensor:
    for x in policy:
        x.grad = None
    exp_ret.sum().backward()

    K_grad, k_grad = (x.grad.detach() for x in policy)
    return linear_feedback_norm((K_grad, k_grad))


def policy_svg_norm(
    policy: lqr.Linear,
    dynamics: lqr.LinSDynamics,
    cost: lqr.QuadCost,
    init: lqr.GaussInit,
) -> tuple[Tensor, Tensor]:
    eret = policy_performance(policy, dynamics, cost, init)
    svg_norm = compute_batched_svg_norm(eret, policy)
    return eret, svg_norm

In [ ]:
def test_random_policy_svg():
    dynamics, cost, init = sample_lqg(n_state=2, n_ctrl=2, horizon=100, n_batch=10)
    policy = tuple(x.requires_grad_(True) for x in sample_policy(dynamics, cost))
    eret, svg_norm = policy_svg_norm(policy, dynamics, cost, init)
    print(f"Expected return: {eret}, ({eret.dtype}, {eret.shape})")
    print(f"SVG norm: {svg_norm} ({svg_norm.dtype}, {svg_norm.shape})")

In [ ]:
test_random_policy_svg()

In [ ]:
def test_optimal_policy_svg():
    dynamics, cost, init = sample_lqg(n_state=2, n_ctrl=2, horizon=100, n_batch=10)
    pistar, _, vstar = solution(dynamics, cost)
    loss_fn = ExpectedValue()
    eret = -loss_fn(init, tuple(x.select("H", 0) for x in vstar))
    print(f"Optimal expected return: {eret} ({eret.dtype})")

    policy = tuple(x.requires_grad_(True) for x in pistar)
    eret = -policy_loss(policy, dynamics, cost, init)
    print(f"Expected return from Prediction algorithm: {eret} ({eret.dtype})")
    svg_norm = compute_batched_svg_norm(eret, policy)
    print(f"Optimal policy SVG norm: {svg_norm} ({svg_norm.dtype}, {svg_norm.shape})")

In [ ]:
test_optimal_policy_svg()

---
## Beyond averages

Next, we evaluate the distribution of optimal/random policy performances for certain LQG problem sizes

### Calculating the expected return (performance) & SVG norm for a batch of LQGs

In [ ]:
def expected_return(init: lqr.GaussInit, vval: lqr.Quadratic) -> Tensor:
    expected_value = ExpectedValue()
    expected_cost = expected_value(init, tuple(x.select("H", 0) for x in vval))
    return -expected_cost

In [ ]:
@torch.no_grad()
def performance_samples(
    dims: tuple[int, int, int],
    optimal: bool = False,
    samples: int = 100,
    clip_ctrl_dim: bool = False,
) -> np.ndarray:
    n_state, n_ctrl, horizon = dims
    if clip_ctrl_dim and n_ctrl < n_state:
        return np.full((samples,), fill_value=np.nan)

    dynamics, cost, init = sample_lqg(n_state, n_ctrl, horizon, n_batch=samples)

    if optimal:
        pistar, qstar, vstar = solution(dynamics, cost)
        eret = expected_return(init, vstar)
    else:
        policy = sample_policy(dynamics, cost)
        eret = policy_performance(policy, dynamics, cost, init)

    return eret.numpy()

In [ ]:
def svg_norm_samples(
    dims: tuple[int, int, int], samples: int = 100, clip_ctrl_dim: bool = False
) -> np.ndarray:
    n_state, n_ctrl, horizon = dims
    if clip_ctrl_dim and n_ctrl < n_state:
        return np.full((samples,), fill_value=np.nan)

    dynamics, cost, init = sample_lqg(n_state, n_ctrl, horizon, n_batch=samples)
    policy = tuple(g.requires_grad_(True) for g in sample_policy(dynamics, cost))
    _, svg_norm = policy_svg_norm(policy, dynamics, cost, init)
    return svg_norm.numpy()

### Plotting the performance distribution

In [ ]:
def plot_optimal_vs_random_performances_hist(
    dims: tuple[int, int, int], samples: int = 1000
):
    optimal = pd.DataFrame(
        {"performance": performance_samples(dims, optimal=True, samples=samples)}
    )
    random = pd.DataFrame(
        {"performance": performance_samples(dims, optimal=False, samples=samples)}
    )

    optimal["optimal"] = True
    random["optimal"] = False
    data = pd.concat([optimal, random])

    sns.histplot(
        data=data,
        x="performance",
        hue="optimal",
        hue_order=[True, False],
        stat="density",
        #         bins=samples // 10
    )

In [ ]:
plot_optimal_vs_random_performances_hist((2, 2, 100))

In [ ]:
plot_optimal_vs_random_performances_hist((2, 2, 1000))

In [ ]:
plot_optimal_vs_random_performances_hist((4, 4, 100))

In [ ]:
plot_optimal_vs_random_performances_hist((10, 10, 100))

In [ ]:
plot_optimal_vs_random_performances_hist((10, 10, 200))

In [ ]:
plot_optimal_vs_random_performances_hist((20, 20, 100))

### Plotting the SVG norm distribution

In [ ]:
def plot_svg_norm_hist(dims: tuple[int, int, int], samples: int = 1000):
    data = pd.DataFrame({"svg norm": svg_norm_samples(dims, samples=samples)})
    sns.histplot(data=data, x="svg norm", stat="density")

In [ ]:
plot_svg_norm_hist((2, 2, 20))

In [ ]:
plot_svg_norm_hist((2, 2, 100))

In [ ]:
plot_svg_norm_hist((4, 4, 20))

In [ ]:
plot_svg_norm_hist((10, 10, 20))

In [ ]:
plot_svg_norm_hist((10, 10, 200))

In [ ]:
plot_svg_norm_hist((20, 20, 100))

---
## Evaluate cost scaling against each variable

### Checklist
- [x] Stack several LQGs including init dists
- [x] Define range of `n_state`s, `n_ctrl`s, and `horizon`s
- [x] Fix initial values of `n_state=2`, `n_ctrl=2`, `horizon=100`
- [x] Iterate over one of the ranges, e.g., `horizon`s:
  - [x] Generate several LQGs with the current `n_state`, `n_ctrl`, `horizon` and stack them
  - [x] Find the optimal solutions (policies) and evaluate the optimal expected returns
  - [x] Average the results accross the LQGs generated
  - [x] Plot the average against the current `n_state`, `n_ctrl`, `horizon`

In [ ]:
def performance_matrix(
    state_dim_arr: np.ndarray,
    ctrl_dim_arr: np.ndarray,
    horizon_arr: np.ndarray,
    perf_sampling_fn: Callable[[int, int, int], np.ndarray],
) -> np.ndarray:
    pbar = tqdm(
        zip(state_dim_arr, ctrl_dim_arr, horizon_arr),
        total=len(ctrl_dim_arr),
        desc="Computing performance by LQG dims",
    )

    exp_ret_arrs = [
        perf_sampling_fn(n_state, n_ctrl, horizon) for n_state, n_ctrl, horizon in pbar
    ]

    exp_ret_mat = np.vstack(exp_ret_arrs)
    return exp_ret_mat

In [ ]:
def average_performance_lineplot(
    state_dim: np.ndarray,
    ctrl_dim: np.ndarray,
    horizon: np.ndarray,
    x_axis: tuple[str, np.ndarray],
):
    x, x_arr = x_axis
    y = "average performance"
    hue = "optimal"
    n_samples = 200

    def sampler(s, c, h, *, optimal: bool):
        return performance_samples((s, c, h), samples=n_samples, optimal=optimal)

    optimal = {
        x: x_arr,
        y: performance_matrix(
            state_dim, ctrl_dim, horizon, partial(sampler, optimal=True)
        ).mean(-1),
        hue: np.ones_like(x_arr, dtype=bool),
    }
    random = {
        x: x_arr,
        y: performance_matrix(
            state_dim, ctrl_dim, horizon, partial(sampler, optimal=False)
        ).mean(-1),
        hue: np.zeros_like(x_arr, dtype=bool),
    }

    data = pd.concat(list(map(pd.DataFrame, (optimal, random))))
    ax = sns.lineplot(x=x, y=y, data=data, hue=hue)
    plt.show()

#### Performance vs. state dimension

In [ ]:
x_axis = np.arange(2, 12)
average_performance_lineplot(
    state_dim=x_axis,
    ctrl_dim=np.full(10, 2, dtype=int),
    horizon=np.full(10, 100, dtype=int),
    x_axis=("state dim", x_axis),
)

#### Performance vs. control dimension

In [ ]:
state_dims = np.full(10, 2, dtype=int)
ctrl_dims = np.arange(2, 12)
horizons = np.full(10, 100, dtype=int)
average_performance_lineplot(
    state_dim=state_dims,
    ctrl_dim=ctrl_dims,
    horizon=horizons,
    x_axis=("ctrl dim", ctrl_dims),
)

#### Performance vs. horizon length

In [ ]:
state_dims = np.full(10, 2, dtype=int)
ctrl_dims = np.full(10, 2, dtype=int)
horizons = 100 * np.arange(1, 11)
average_performance_lineplot(state_dims, ctrl_dims, horizons, ("horizon", horizons))

#### Performance vs. state & control dimension

In [ ]:
state_dims = ctrl_dims = np.arange(1, 21) + 1
horizons = np.full(20, 100, dtype=int)
average_performance_lineplot(
    state_dims, ctrl_dims, horizons, ("state & ctrl dim", state_dims)
)

---
## Multivariate analysis of cost scaling

### Checklist
- [x] Plot the average expected return against two variables, e.g., `n_state` and `n_ctrl`, as a 3D surface plot
- [ ] ~Use the insights gained from observing the cost scaling to propose a _downscaling_ parameter for random QuadCosts~
- [x] Use the analysis above to also derive a scale parameter for the random Gaussian noise for policy generation

In [ ]:
def large_fig(*args, scale: float = 2.0, **kwargs):
    return plt.figure(*args, figsize=[scale * 6.4, scale * 4.8], **kwargs)

In [ ]:
def plot_3d_from_2d_function(
    ax,
    xrange: np.ndarray,
    yrange: np.ndarray,
    function: callable[[np.ndarray, np.ndarray], np.ndarray],
):
    length = len(xrange)
    xbatch, ybatch = np.meshgrid(xrange, yrange)
    assert xbatch.shape == ybatch.shape == (length, length)

    x_arr, y_arr = map(lambda x: x.reshape((-1,)), (xbatch, ybatch))
    assert (
        x_arr.shape == y_arr.shape == (length ** 2,)
    ), f"expected: {(length ** 2,)}; x_arr: {x_arr.shape}; y_arr {y_arr.shape}"

    z_arr = function(x_arr, y_arr)
    assert z_arr.shape == x_arr.shape, f"{z_arr.shape} != {x_arr.shape}"

    zbatch = z_arr.reshape(xbatch.shape)
    ax.plot_surface(xbatch, ybatch, zbatch, cmap=cm.coolwarm)

### State vs. control dimension with fixed horizon

In what follows we set the horizon length to 100 steps.

#### Best expected return

In [ ]:
def performances_by_state_ctrl_dim(
    state_dim_arr: np.ndarray, ctrl_dim_arr: np.ndarray, horizon: int, optimal: bool
) -> np.ndarray:
    horizon_arr = np.full_like(state_dim_arr, fill_value=horizon)
    exp_opt_ret_mat = performance_matrix(
        state_dim_arr,
        ctrl_dim_arr,
        horizon_arr,
        lambda s, c, h: performance_samples(
            (s, c, h), optimal=optimal, samples=100, clip_ctrl_dim=False
        ),
    )
    return exp_opt_ret_mat

In [ ]:
def average_performance_by_state_ctrl_dim(
    state_dim_arr: np.ndarray, ctrl_dim_arr: np.ndarray, optimal: bool
) -> np.ndarray:
    z_arr = performances_by_state_ctrl_dim(
        state_dim_arr, ctrl_dim_arr, horizon=100, optimal=optimal
    ).mean(axis=-1)
    # Filtering NaNs
    z_arr = np.nan_to_num(z_arr, nan=np.nanmin(z_arr))
    return z_arr

In [ ]:
fig = large_fig()
ax = fig.add_subplot(projection="3d")
plot_3d_from_2d_function(
    ax,
    xrange=np.arange(1, 21) + 1,
    yrange=np.arange(1, 21) + 1,
    function=lambda x, y: average_performance_by_state_ctrl_dim(x, y, optimal=True),
)
ax.set_xlabel("state dim")
ax.set_ylabel("ctrl dim")
ax.set_title("average best performance")
plt.show()

#### Expected return from random policy

In [ ]:
fig = large_fig()
ax = fig.add_subplot(projection="3d")
plot_3d_from_2d_function(
    ax,
    xrange=np.arange(1, 21) + 1,
    yrange=np.arange(1, 21) + 1,
    function=lambda x, y: average_performance_by_state_ctrl_dim(x, y, optimal=False),
)
ax.set_xlabel("state dim")
ax.set_ylabel("ctrl dim")
ax.set_title("average random performance")
plt.show()

### State & control dimension vs. horizon length

#### Best expected return

In [ ]:
def performances_by_horizon(
    state_ctrl_dim: np.ndarray, horizon_arr: np.ndarray, optimal: bool
) -> np.ndarray:
    state_dim_arr = ctrl_dim_arr = state_ctrl_dim
    opt_perf_mat = performance_matrix(
        state_dim_arr,
        ctrl_dim_arr,
        horizon_arr,
        lambda s, c, h: performance_samples((s, c, h), optimal=optimal, samples=100),
    )
    return opt_perf_mat

In [ ]:
def average_performance_by_horizon(
    state_ctrl_dim: np.ndarray, horizon_arr: np.ndarray, optimal: bool
) -> np.ndarray:
    z_arr = performances_by_horizon(state_ctrl_dim, horizon_arr, optimal=optimal).mean(
        axis=-1
    )
    # Filtering NaNs
    z_arr = np.nan_to_num(z_arr, nan=np.nanmin(z_arr))
    return z_arr

In [ ]:
fig = large_fig()
ax = fig.add_subplot(projection="3d")
plot_3d_from_2d_function(
    ax,
    xrange=np.arange(1, 21) + 1,
    yrange=100 + 10 * np.arange(1, 21),
    function=lambda x, y: average_performance_by_horizon(x, y, optimal=True),
)
ax.set_xlabel("state/ctrl dim")
ax.set_ylabel("horizon")
ax.set_title("average optimal performance")
plt.show()

#### Random expected return

In [ ]:
fig = large_fig()
ax = fig.add_subplot(projection="3d")
plot_3d_from_2d_function(
    ax,
    xrange=np.arange(1, 21) + 1,
    yrange=100 + 10 * np.arange(1, 21),
    function=lambda x, y: average_performance_by_horizon(x, y, optimal=False),
)
ax.set_xlabel("state/ctrl dim")
ax.set_ylabel("horizon")
ax.set_title("average random performance")
plt.show()